In [1]:
import csv

from sklearn.linear_model import ElasticNet
import numpy as np
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

TEST_DATA_FILE_1 = "test_data_1.csv"
TEST_DATA_FILE_2 = "test_data_2.csv"
TRAIN_DATA_FILE_1 = "train_data_1.csv"
TRAIN_DATA_FILE_2 = "train_data_2.csv"
PROD_INDEX = 6
DIR_INDEX = 5
ERROR_THRESHOLD_1 = 0
ERROR_THRESHOLD_2 = 0.2
ERROR_THRESHOLD_3 = 0.4

In [2]:
def read_data_file(filename):    
    data_file = file(filename, 'r')
    data = csv.reader(data_file, delimiter=',')
    labels = []
    values = []
    for row in data:
        value = []
        prod_value = 0
        writer_value = 0
        for index in range(1,PROD_INDEX):
            value.append(float(row[index]))
        for index in range(PROD_INDEX + 1,PROD_INDEX + int(row[PROD_INDEX]) + 1):
            prod_value = max(float(row[index]),prod_value)
        value.append(prod_value)
        WRITER_INDEX = PROD_INDEX + int(row[PROD_INDEX]) + 1
        for index in range(WRITER_INDEX+1,len(row)):
            writer_value = max(float(row[index]),writer_value)
        value.append(writer_value)
        labels.append(float(row[0]))
        values.append(value)
    print 'Done Reading File',filename
    return labels,values

In [3]:
train_data_1_labels, train_data_1_values = read_data_file(TRAIN_DATA_FILE_1)
train_data_2_labels, train_data_2_values = read_data_file(TRAIN_DATA_FILE_2)

test_data_1_labels, test_data_1_values = read_data_file(TEST_DATA_FILE_1)
test_data_2_labels, test_data_2_values = read_data_file(TEST_DATA_FILE_2)

Done Reading File train_data_1.csv
Done Reading File train_data_2.csv
Done Reading File test_data_1.csv
Done Reading File test_data_2.csv


In [4]:
def get_error_metric(threshold,actual_labels,predicted_labels):
    error_count = 0
    errors = np.zeros(len(actual_labels))
    for index in range(0,len(actual_labels)):
        errors[index] = (abs(float(actual_labels[index])-float(predicted_labels[index])))
        if abs(float(actual_labels[index])-float(predicted_labels[index])) > threshold:
            error_count +=1
    return error_count*100/len(actual_labels),np.mean(errors),np.std(errors)

def get_mean_squared_error(actual_labels,predicted_labels):
    return mean_squared_error(actual_labels,predicted_labels)

In [5]:
def get_predictions(classifier,val,data):
    if val==1:
        X = np.asarray(train_data_1_values)
        Y = np.asarray(train_data_1_labels)
    else:
        X = np.asarray(train_data_2_values)
        Y = np.asarray(train_data_2_labels)
    classifier.fit(X,Y)
    return classifier.predict(data)

In [6]:
classifier = ElasticNet()
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

Lambda:1
Errors for Mechanism-1: Zero Tolerance: 100 0.578906615082 0.453356999044
Errors for Mechanism-1: 0.1 Tolerance: 78 0.578906615082 0.453356999044
Errors for Mechanism-1: 0.2 Tolerance: 58 0.578906615082 0.453356999044
MSE Mechansim-1: 0.540665437567


In [7]:
classifier = ElasticNet(alpha=0.1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

classifier = ElasticNet(alpha=-1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

Lambda:0.1
Errors for Mechanism-1: Zero Tolerance: 100 0.28135262735 0.280544236485
Errors for Mechanism-1: 0.1 Tolerance: 49 0.28135262735 0.280544236485
Errors for Mechanism-1: 0.2 Tolerance: 24 0.28135262735 0.280544236485
MSE Mechansim-1: 0.157864369542
Lambda:-1
Errors for Mechanism-1: Zero Tolerance: 100 0.850173415149 0.660573005044
Errors for Mechanism-1: 0.1 Tolerance: 85 0.850173415149 0.660573005044
Errors for Mechanism-1: 0.2 Tolerance: 70 0.850173415149 0.660573005044
MSE Mechansim-1: 1.15915153082


In [8]:
classifier = ElasticNet(alpha=0.01)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.01'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

classifier = ElasticNet(alpha=2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)    

Lambda:0.01
Errors for Mechanism-1: Zero Tolerance: 100 0.280528879312 0.272417264479
Errors for Mechanism-1: 0.1 Tolerance: 49 0.280528879312 0.272417264479
Errors for Mechanism-1: 0.2 Tolerance: 24 0.280528879312 0.272417264479
MSE Mechansim-1: 0.152907618114
Lambda:2
Errors for Mechanism-1: Zero Tolerance: 100 0.850173415149 0.660573005044
Errors for Mechanism-1: 0.1 Tolerance: 85 0.850173415149 0.660573005044
Errors for Mechanism-1: 0.2 Tolerance: 70 0.850173415149 0.660573005044
MSE Mechansim-1: 1.15915153082


In [9]:
classifier = ElasticNet(alpha=-2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

Lambda:-2
Errors for Mechanism-1: Zero Tolerance: 100 0.850173415149 0.660573005044
Errors for Mechanism-1: 0.1 Tolerance: 85 0.850173415149 0.660573005044
Errors for Mechanism-1: 0.2 Tolerance: 70 0.850173415149 0.660573005044
MSE Mechansim-1: 1.15915153082


In [10]:
classifier = ElasticNet()
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:1
Errors for Mechanism-2: Zero Tolerance: 100 0.639909654312 0.529049980337
Errors for Mechanism-2: 0.1 Tolerance: 79 0.639909654312 0.529049980337
Errors for Mechanism-2: 0.2 Tolerance: 60 0.639909654312 0.529049980337
MSE Mechansim-2: 0.689378247377


In [11]:
classifier = ElasticNet(alpha=-1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)       

Lambda:-1
Errors for Mechanism-2: Zero Tolerance: 100 0.867761932636 0.71859028245
Errors for Mechanism-2: 0.1 Tolerance: 84 0.867761932636 0.71859028245
Errors for Mechanism-2: 0.2 Tolerance: 69 0.867761932636 0.71859028245
MSE Mechansim-2: 1.26938276576


In [12]:
classifier = ElasticNet(alpha=2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)  

classifier = ElasticNet(alpha=-2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print 'MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)


Lambda:2
Errors for Mechanism-2: Zero Tolerance: 100 0.867761932636 0.71859028245
Errors for Mechanism-2: 0.1 Tolerance: 84 0.867761932636 0.71859028245
Errors for Mechanism-2: 0.2 Tolerance: 69 0.867761932636 0.71859028245
MSE Mechansim-2: 1.26938276576
Lambda:-2
Errors for Mechanism-2: Zero Tolerance: 100 0.867761932636 0.71859028245
Errors for Mechanism-2: 0.1 Tolerance: 84 0.867761932636 0.71859028245
Errors for Mechanism-2: 0.2 Tolerance: 69 0.867761932636 0.71859028245
MSE Mechansim-2: 1.26938276576


In [13]:

avg_rating = 6.031

def read_data_file_2(filename):    
    data_file = file(filename, 'r')
    data = csv.reader(data_file, delimiter=',')
    labels = []
    values = []
    for row in data:
        value = []
        prod_value = 0
        writer_value = 0
        for index in range(1,PROD_INDEX):
            value.append(float(row[index]))
        for index in range(PROD_INDEX + 1,PROD_INDEX + int(row[PROD_INDEX]) + 1):
            prod_value = max(float(row[index]),prod_value)
        value.append(prod_value)
        WRITER_INDEX = PROD_INDEX + int(row[PROD_INDEX]) + 1
        for index in range(WRITER_INDEX+1,len(row)):
            writer_value = max(float(row[index]),writer_value)
        value.append(writer_value)
        labels.append(float(row[0])-avg_rating)
        values.append(value)
    print 'Done Reading File',filename
    return labels,values

train_data_1_labels, train_data_1_values = read_data_file_2(TRAIN_DATA_FILE_1)
train_data_2_labels, train_data_2_values = read_data_file_2(TRAIN_DATA_FILE_2)

test_data_1_labels, test_data_1_values = read_data_file_2(TEST_DATA_FILE_1)
test_data_2_labels, test_data_2_values = read_data_file_2(TEST_DATA_FILE_2)

Done Reading File train_data_1.csv
Done Reading File train_data_2.csv
Done Reading File test_data_1.csv
Done Reading File test_data_2.csv


In [14]:
classifier = ElasticNet()
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)


Lambda:1
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.578906615082 0.453356999044
Errors(2) for Mechanism-1: 0.1 Tolerance: 78 0.578906615082 0.453356999044
Errors(2) for Mechanism-1: 0.2 Tolerance: 58 0.578906615082 0.453356999044
(2) MSE Mechansim-1: 0.540665437567


In [15]:
classifier = ElasticNet(alpha=0.01)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.01'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

classifier = ElasticNet(alpha=-1)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

Lambda:0.01
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.280528879312 0.272417264479
Errors(2) for Mechanism-1: 0.1 Tolerance: 49 0.280528879312 0.272417264479
Errors(2) for Mechanism-1: 0.2 Tolerance: 24 0.280528879312 0.272417264479
(2) MSE Mechansim-1: 0.152907618114
Lambda:-1
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.850173415149 0.660573005044
Errors(2) for Mechanism-1: 0.1 Tolerance: 85 0.850173415149 0.660573005044
Errors(2) for Mechanism-1: 0.2 Tolerance: 70 0.850173415149 0.660573005044
(2) MSE Mechansim-1: 1.15915153082


In [16]:
classifier = ElasticNet(alpha=0.001)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:0.001'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

classifier = ElasticNet(alpha=2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)


Lambda:0.001
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.280909486191 0.271944778358
Errors(2) for Mechanism-1: 0.1 Tolerance: 50 0.280909486191 0.271944778358
Errors(2) for Mechanism-1: 0.2 Tolerance: 24 0.280909486191 0.271944778358
(2) MSE Mechansim-1: 0.152864101908
Lambda:2
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.850173415149 0.660573005044
Errors(2) for Mechanism-1: 0.1 Tolerance: 85 0.850173415149 0.660573005044
Errors(2) for Mechanism-1: 0.2 Tolerance: 70 0.850173415149 0.660573005044
(2) MSE Mechansim-1: 1.15915153082


In [17]:
classifier = ElasticNet(alpha=-2)
obs_labels = get_predictions(classifier,1,test_data_1_values)

print 'Lambda:-2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_1_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_1_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_1_labels,obs_labels)
print 'Errors(2) for Mechanism-1: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-1: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-1: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-1:',get_mean_squared_error(test_data_1_labels,obs_labels)

Lambda:-2
Errors(2) for Mechanism-1: Zero Tolerance: 100 0.850173415149 0.660573005044
Errors(2) for Mechanism-1: 0.1 Tolerance: 85 0.850173415149 0.660573005044
Errors(2) for Mechanism-1: 0.2 Tolerance: 70 0.850173415149 0.660573005044
(2) MSE Mechansim-1: 1.15915153082


In [18]:
classifier = ElasticNet()
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.639909654312 0.529049980337
Errors(2) for Mechanism-2: 0.1 Tolerance: 79 0.639909654312 0.529049980337
Errors(2) for Mechanism-2: 0.2 Tolerance: 60 0.639909654312 0.529049980337
(2) MSE Mechansim-2: 0.689378247377


In [19]:
classifier = ElasticNet(alpha=0.1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

classifier = ElasticNet(alpha=-1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

classifier = ElasticNet(alpha=0.01)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.01'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)


Lambda:0.1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.302415530563 0.300053606614
Errors(2) for Mechanism-2: 0.1 Tolerance: 51 0.302415530563 0.300053606614
Errors(2) for Mechanism-2: 0.2 Tolerance: 26 0.302415530563 0.300053606614
(2) MSE Mechansim-2: 0.181487319968
Lambda:-1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.867761932636 0.71859028245
Errors(2) for Mechanism-2: 0.1 Tolerance: 84 0.867761932636 0.71859028245
Errors(2) for Mechanism-2: 0.2 Tolerance: 69 0.867761932636 0.71859028245
(2) MSE Mechansim-2: 1.26938276576
Lambda:0.01
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.300192184168 0.279964672727
Errors(2) for Mechanism-2: 0.1 Tolerance: 55 0.300192184168 0.279964672727
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.300192184168 0.279964672727
(2) MSE Mechansim-2: 0.16849556541


In [27]:
classifier = ElasticNet(alpha=2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

classifier = ElasticNet(alpha=0.02)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.02'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)



Lambda:2
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.867761932636 0.71859028245
Errors(2) for Mechanism-2: 0.1 Tolerance: 84 0.867761932636 0.71859028245
Errors(2) for Mechanism-2: 0.2 Tolerance: 69 0.867761932636 0.71859028245
(2) MSE Mechansim-2: 1.26938276576
Lambda:0.02
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.299361615553 0.28145764266
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.299361615553 0.28145764266
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.299361615553 0.28145764266
(2) MSE Mechansim-2: 0.168835781478


In [21]:
classifier = ElasticNet(alpha=-2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:-2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:-2
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.867761932636 0.71859028245
Errors(2) for Mechanism-2: 0.1 Tolerance: 84 0.867761932636 0.71859028245
Errors(2) for Mechanism-2: 0.2 Tolerance: 69 0.867761932636 0.71859028245
(2) MSE Mechansim-2: 1.26938276576


In [22]:
classifier = ElasticNet(alpha=0.5)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.5'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:0.5
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.438760950643 0.383199740054
Errors(2) for Mechanism-2: 0.1 Tolerance: 68 0.438760950643 0.383199740054
Errors(2) for Mechanism-2: 0.2 Tolerance: 43 0.438760950643 0.383199740054
(2) MSE Mechansim-2: 0.339353212586


In [23]:
classifier = ElasticNet(alpha=0.2)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.2'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:0.2
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.323581175132 0.326828732321
Errors(2) for Mechanism-2: 0.1 Tolerance: 53 0.323581175132 0.326828732321
Errors(2) for Mechanism-2: 0.2 Tolerance: 29 0.323581175132 0.326828732321
(2) MSE Mechansim-2: 0.21152179717


In [24]:
classifier = ElasticNet(alpha=0.1)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.1'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:0.1
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.302415530563 0.300053606614
Errors(2) for Mechanism-2: 0.1 Tolerance: 51 0.302415530563 0.300053606614
Errors(2) for Mechanism-2: 0.2 Tolerance: 26 0.302415530563 0.300053606614
(2) MSE Mechansim-2: 0.181487319968


In [25]:
classifier = ElasticNet(alpha=0.02)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.02'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:0.02
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.299361615553 0.28145764266
Errors(2) for Mechanism-2: 0.1 Tolerance: 54 0.299361615553 0.28145764266
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.299361615553 0.28145764266
(2) MSE Mechansim-2: 0.168835781478


In [26]:
classifier = ElasticNet(alpha=0.002)
obs_labels = get_predictions(classifier,2,test_data_2_values)

print 'Lambda:0.002'
error_1,mean_1,std_1 = get_error_metric(ERROR_THRESHOLD_1,test_data_2_labels,obs_labels)
error_2,mean_2,std_2 = get_error_metric(ERROR_THRESHOLD_2,test_data_2_labels,obs_labels)
error_3,mean_3,std_3 = get_error_metric(ERROR_THRESHOLD_3,test_data_2_labels,obs_labels)
print 'Errors(2) for Mechanism-2: Zero Tolerance:',error_1,mean_1,std_1
print 'Errors(2) for Mechanism-2: 0.1 Tolerance:',error_2,mean_2,std_2
print 'Errors(2) for Mechanism-2: 0.2 Tolerance:',error_3,mean_3,std_3
print '(2) MSE Mechansim-2:',get_mean_squared_error(test_data_2_labels,obs_labels)

Lambda:0.002
Errors(2) for Mechanism-2: Zero Tolerance: 100 0.301064252219 0.279012186783
Errors(2) for Mechanism-2: 0.1 Tolerance: 55 0.301064252219 0.279012186783
Errors(2) for Mechanism-2: 0.2 Tolerance: 25 0.301064252219 0.279012186783
(2) MSE Mechansim-2: 0.168487484338
